# Assignment 6: Implement a Breadth-first Search Algorithm


In this exercise, we will begin to explore the concept of hashing and how it related to various object containers with respect to computational complexity.  We will begin with the base code for as described in Chapter 5 of Grokking Algorithms (Bhargava 2016).  


## Deliverables:

 

    1) Create a five level 'connections' graph (all unique, no duplicates).
    
        * The first level should have five people and each person in first level should have five connections
          (they can be exclusive to one person or shared with other members on the same level, but all five should be unique, no duplicates)
          Each person in the second level should have five connections (they can be exclusive to one person or shared with other members on the same level, but all five should be unique, no duplicates).

        * Continue the same for third, fourth, and fifth levels
        * There should be no reverse connections (i.e. first level can connect to second level but second cannot have any reverse connection with the first, and so on for 3rd, 4th, and 5th levels)

    2) Now search for three people on each level and capture the time for each search.
    
    3) Organize the results of the study into an table showing the 'name' that you searched for, the level of the search (first, second, third, fourth, fifth), and the time it took.
       Discuss any findings.

    4) Using the average time for each level, create a plot and compare times at each level (five bars or any other suitable visual).  Discuss any findings.


    
### Prepare an executive summary of your results, referring to the table and figures you have generated. Explain how your results relate to big O notation. Describe your results in language that management can understand. This summary should be included as text paragraphs in the Jupyter notebook. Explain how the algorithm works and why it is a useful to data engineers.

# A. Setup: Library imports, Function construction and Array generation

In [13]:
import numpy as np
import pandas as pd

import seaborn as sns
import time

import random
import string


RANDOM_SEED = 8 #sets random seed

In [ ]:
from collections import deque

def person_is_seller(name):
      return name[-1] == 'm'

graph = {}
graph["you"] = ["alice", "bob", "claire"]
graph["bob"] = ["anuj", "peggy"]
graph["alice"] = ["peggy"]
graph["claire"] = ["thom", "jonny"]
graph["anuj"] = []
graph["peggy"] = []
graph["thom"] = []
graph["jonny"] = []

def search(name):
    search_queue = deque()
    search_queue += graph[name]
    # This is how you keep track of which people you've searched before.
    searched = set()
    while search_queue:
        person = search_queue.popleft()
        # Only search this person if you haven't already searched them.
        if person not in searched:
            if person_is_seller(person):
                print(person + " is a mango seller!")
                return True
            else:
                search_queue += graph[person]
                # Marks this person as searched
                searched.add(person)
    return False

search("you")

In [ ]:
graph = {}

#first levels
graph['Alfred'] = ['Alice', 'Ahmed','Brian', 'Barbara', 'Bart']
graph['Alice'] = ['Alfred', 'Anton', 'Brian', 'Bob', 'Beatrice']
graph['Anton'] = ['Alice', 'Beatrice', 'Bob', 'Bart', 'Barbara']
graph['Arthur'] = ['Ahmed', 'Brian', 'Bob',  'Bart', 'Cara']
graph['Ahmed'] =['Alfred', 'Arthur' , 'Dory', 'Evelyn', 'Edward']

#second levels
graph['Beatrice'] = ['Brian', 'Bart','Cara' ,'Claire', 'Chuck' ]
graph['Brian'] = ['Beatrice', 'Bob', 'Chelsey', 'Chuck' , 'Charles']
graph['Bob'] = ['Brian', 'Barbara', 'Claire', 'Dory' , 'Daryl' ]
graph['Bart'] = ['Beatrice', 'Chelsey' , 'Chuck' , 'Charles' , 'Denise' ]
graph['Barbara'] = ['Bob', 'Derek' , 'Denise' , 'Daryl', 'Ester' ]

#third levels
graph['Chelsey'] = ['Cara', 'Chuck', 'Dory', 'Dennis', 'Edward' ]
graph['Cara'] = ['Chelsey', 'Claire', 'Charles', 'Derek','Dennis']
graph['Claire'] = ['Cara', 'Chuck', 'Dory', 'Daryl', 'Evelyn']
graph['Chuck'] = ['Claire', 'Chelsey', 'Derek', 'Dennis', 'Daryl']
graph['Charles'] = ['Cara', 'Claire', 'Denise', 'Dory', 'Dennis']

#forth levels
graph['Denise'] =
graph['Derek'] =
graph['Dory'] =
graph['Dennis'] =
graph['Daryl'] =

#fifth levels
graph['Eugene']  = ['Edward',]
graph['Edward'] = ['Eugene','Ester']
graph['Elizibeth'] = []
graph['Ester'] = ['Edward']
graph['Evelyn'] =